In [1]:
!pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-g2axy5op
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-g2axy5op
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 112.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 114.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 83.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 218.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 224.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 427.6 MB/s eta 0:00:00
   

In [2]:
!spacy download es_core_news_sm
import spacy
from spacy.lang.es.examples import sentences

nlp = spacy.load("es_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 54.8 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [6]:
base_dir = 'data'

In [7]:
import os
import re
import pandas as pd
import spacy
from tqdm import tqdm
import pke
from pke.lang import stopwords


# --- Tokenizer fix para palabras con guiones ---
from spacy.tokenizer import _get_regex_pattern
re_token_match = _get_regex_pattern(nlp.Defaults.token_match)
re_token_match = f"({re_token_match}|\w+-\w+)"
nlp.tokenizer.token_match = re.compile(re_token_match).match

# Ruta base de los artículos
os.makedirs(base_dir, exist_ok=True)

# Diccionario para almacenar resultados de YAKE
resultados_yake = {}

# --- Extracción de términos con YAKE ---
print("🔍 Extrayendo términos con YAKE...")
for i in range(1, 41):
    carpeta = os.path.join(base_dir, f"articulo_{i}")
    archivo_txt = os.path.join(carpeta, f"articulo_{i}.txt")

    if not os.path.exists(archivo_txt):
        print(f"⚠️ Archivo no encontrado: {archivo_txt}")
        continue

    try:
        with open(archivo_txt, "r", encoding="utf-8") as f:
            texto = f.read().strip()

        if not texto:
            print(f"⚠️ Artículo {i} vacío.")
            continue

        extractor = pke.unsupervised.YAKE()
        extractor.load_document(input=texto,
                                language='es',
                                stoplist=stopwords.get('es'),
                                normalization=None)

        extractor.candidate_selection(n=3)
        extractor.candidate_weighting(window=2, use_stems=False)

        # Obtener todos los candidatos ordenados por puntuación
        keyphrases = sorted(extractor.weights.items(), key=lambda x: x[1])
        resultados_yake[f"articulo_{i}"] = keyphrases

        # Guardar en archivo para evaluación
        path_out = os.path.join(carpeta, 'terminos_extraidos_yake.txt')
        with open(path_out, 'w', encoding='utf-8') as f_out:
            for termino, _ in keyphrases:
                f_out.write(termino.strip().lower() + '\n')

        print(f"✅ Artículo {i}: {len(keyphrases)} términos extraídos")

    except Exception as e:
        print(f"❌ Error procesando artículo {i}: {e}")

🔍 Extrayendo términos con YAKE...
✅ Artículo 1: 247 términos extraídos
✅ Artículo 2: 112 términos extraídos
✅ Artículo 3: 121 términos extraídos
✅ Artículo 4: 157 términos extraídos
✅ Artículo 5: 50 términos extraídos
✅ Artículo 6: 68 términos extraídos
✅ Artículo 7: 57 términos extraídos
✅ Artículo 8: 258 términos extraídos
✅ Artículo 9: 80 términos extraídos
✅ Artículo 10: 42 términos extraídos
✅ Artículo 11: 721 términos extraídos
✅ Artículo 12: 564 términos extraídos
✅ Artículo 13: 17 términos extraídos
✅ Artículo 14: 128 términos extraídos
✅ Artículo 15: 494 términos extraídos
✅ Artículo 16: 327 términos extraídos
✅ Artículo 17: 242 términos extraídos
✅ Artículo 18: 32 términos extraídos
✅ Artículo 19: 164 términos extraídos
✅ Artículo 20: 129 términos extraídos
✅ Artículo 21: 104 términos extraídos
✅ Artículo 22: 91 términos extraídos
✅ Artículo 23: 143 términos extraídos
✅ Artículo 24: 58 términos extraídos
✅ Artículo 25: 39 términos extraídos
✅ Artículo 26: 159 términos extraíd

In [8]:
# --- Funciones para evaluación ---
def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = list(range(len(s2) + 1))
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def levenshtein_distance_words(s1, s2):
    words1 = s1.split()
    words2 = s2.split()
    if len(words1) < len(words2):
        return levenshtein_distance_words(s2, s1)
    if len(words2) == 0:
        return len(words1)
    previous_row = list(range(len(words2) + 1))
    for i, w1 in enumerate(words1):
        current_row = [i + 1]
        for j, w2 in enumerate(words2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (w1 != w2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def normalized_levenshtein(s1, s2):
    if not s1 and not s2:
        return 0.0
    if " " in s1 or " " in s2:
        dist = levenshtein_distance_words(s1, s2)
        max_len = max(len(s1.split()), len(s2.split()))
    else:
        dist = levenshtein_distance(s1, s2)
        max_len = max(len(s1), len(s2))
    return dist / max_len if max_len else 0.0

def evaluate_annotations(predictions, references):
    TP_exact, TP_partial, FP, FN = 0, 0, 0, 0
    partial_distances = []
    matched_preds = set()
    matched_refs = set()

    for i, pred in enumerate(predictions):
        matched = False
        for j, ref in enumerate(references):
            if ref == pred:
                TP_exact += 1
                matched_preds.add(i)
                matched_refs.add(j)
                matched = True
                break
        if not matched:
            for j, ref in enumerate(references):
                if j in matched_refs:
                    continue
                if pred in ref or ref in pred:
                    TP_partial += 1
                    matched_preds.add(i)
                    matched_refs.add(j)
                    partial_distances.append(normalized_levenshtein(pred, ref))
                    break

    FP = len(predictions) - len(matched_preds)
    FN = len(references) - len(matched_refs)
    return {
        'TP_exact': TP_exact,
        'TP_partial': TP_partial,
        'FP': FP,
        'FN': FN,
        'partial_distances': partial_distances
    }

# --- Evaluación ---
print("\n📊 Evaluando términos extraídos con YAKE...")
resultados = []
global_counts = {'TP_exact': 0, 'TP_partial': 0, 'FP': 0, 'FN': 0, 'partial_distances': []}

for i in range(1, 41):
    carpeta = f'articulo_{i}'
    carpeta_path = os.path.join(base_dir, carpeta)
    path_expert = os.path.join(carpeta_path, 'terminos_validados_todos.txt')
    path_model = os.path.join(carpeta_path, 'terminos_extraidos_yake.txt')

    if not os.path.exists(path_expert) or not os.path.exists(path_model):
        print(f"⚠️ Archivos faltantes en {carpeta}")
        continue

    with open(path_expert, 'r', encoding='utf-8') as f:
        expert_terms = [line.strip().lower() for line in f if line.strip()]

    with open(path_model, 'r', encoding='utf-8') as f:
        candidate_terms = [line.strip().lower() for line in f if line.strip()]

    r = evaluate_annotations(candidate_terms, expert_terms)
    TP_total = r['TP_exact'] + r['TP_partial']
    precision = TP_total / (TP_total + r['FP']) if TP_total + r['FP'] > 0 else 0.0
    recall = TP_total / (TP_total + r['FN']) if TP_total + r['FN'] > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0
    f2 = 5 * precision * recall / (4 * precision + recall) if precision + recall > 0 else 0.0
    avg_lev = sum(r['partial_distances']) / len(r['partial_distances']) if r['partial_distances'] else None

    resultados.append({
        'Artículo': carpeta,
        'TP_exact': r['TP_exact'],
        'TP_partial': r['TP_partial'],
        'FP': r['FP'],
        'FN': r['FN'],
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'F2': f2,
        'Avg_Norm_Levenshtein': avg_lev
    })

    for k in ['TP_exact', 'TP_partial', 'FP', 'FN']:
        global_counts[k] += r[k]
    global_counts['partial_distances'].extend(r['partial_distances'])

# Resultados globales
TP_total = global_counts['TP_exact'] + global_counts['TP_partial']
FP = global_counts['FP']
FN = global_counts['FN']
precision = TP_total / (TP_total + FP) if TP_total + FP > 0 else 0.0
recall = TP_total / (TP_total + FN) if TP_total + FN > 0 else 0.0
f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0
f2 = 5 * precision * recall / (4 * precision + recall) if precision + recall > 0 else 0.0
avg_lev = sum(global_counts['partial_distances']) / len(global_counts['partial_distances']) if global_counts['partial_distances'] else None

# Exportar resultados
ruta_resultados_por_articulo = os.path.join(base_dir, 'evaluacion_anotaciones_por_articulo_yake.csv')
ruta_resultados_global = os.path.join(base_dir, 'evaluacion_anotaciones_global_yake.csv')

try:
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv(ruta_resultados_por_articulo, index=False, encoding='utf-8')
    print(f"📄 Resultados por artículo guardados en: {ruta_resultados_por_articulo}")
except Exception as e:
    print(f"❌ Error al guardar resultados por artículo: {e}")

try:
    df_global = pd.DataFrame([{
        'TP_exact': global_counts['TP_exact'],
        'TP_partial': global_counts['TP_partial'],
        'FP': FP,
        'FN': FN,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'F2': f2,
        'Avg_Norm_Levenshtein': avg_lev
    }])
    df_global.to_csv(ruta_resultados_global, index=False, encoding='utf-8')
    print(f"📄 Resultados globales guardados en: {ruta_resultados_global}")
except Exception as e:
    print(f"❌ Error al guardar resultados globales: {e}")

print("\n✅ Evaluación completada.")


📊 Evaluando términos extraídos con YAKE...
📄 Resultados por artículo guardados en: data/evaluacion_anotaciones_por_articulo_yake.csv
📄 Resultados globales guardados en: data/evaluacion_anotaciones_global_yake.csv

✅ Evaluación completada.
